In [1]:
import pandas as pd
from sqlite3 import connect
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import batch_normalization
from tensorflow.keras.metrics import CategoricalAccuracy, AUC
from tensorflow_addons.metrics import F1Score   




In [2]:
conn = connect("data/dataset_review.db")
dataset = pd.read_sql("SELECT * FROM games", conn)
conn.close()

dataset.set_index("index", inplace=True)
dataset = dataset.loc[dataset["summary"].notnull()]
dataset = dataset.loc[dataset["reviews"].notnull()]
dataset = dataset[dataset["user_review"] != -1]
dataset.reset_index(drop=True, inplace=True)

In [3]:
# tfv = CountVectorizer(analyzer='word',stop_words = 'english', lowercase=True)
# tfv.fit(dataset["summary"])
# summary = tfv.transform(dataset["summary"])
# kmeans = KMeans(n_clusters=30, random_state=7).fit(summary)
# summary_kmeans = np.array(kmeans.labels_)
# np.save("data/summary_kmeans.npy", summary_kmeans)

summary_kmeans = np.load("data/summary_kmeans.npy", allow_pickle=True)

In [5]:
categorical_labels = to_categorical(summary_kmeans, num_classes=30)
dataset = pd.concat([dataset, pd.DataFrame(categorical_labels, columns=["summary_kmean_" + str(i) for i in range(30)])], axis=1)
y = pd.get_dummies((dataset["user_review"] * 10).apply(np.floor, axis=1), prefix='label')
x = dataset.drop(columns=["user_review"])

In [6]:
count_vec = CountVectorizer(analyzer='word', stop_words = 'english')
word_vecs = count_vec.fit_transform(dataset["reviews"].values)

In [7]:
sc = preprocessing.StandardScaler(with_mean=False)
svd = decomposition.TruncatedSVD(n_components=700)
svd_word_vecs = svd.fit_transform(sc.fit_transform(word_vecs))

x.drop(columns=["name", "summary", "reviews"], inplace=True)

x = pd.concat([x, pd.DataFrame(svd_word_vecs), pd.get_dummies(x.platform, prefix='plat')], axis=1)
x.drop(columns=["platform", "publish_year", "publish_month"], inplace=True)

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, random_state=7, test_size=0.2, shuffle=True)

In [17]:
model = Sequential()

model.add(Dense(300, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(batch_normalization.BatchNormalization())


model.add(Dense(len(y.columns)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[CategoricalAccuracy(), F1Score(len(y.columns))])

In [18]:
model.fit(x_train, y=y_train, batch_size=64, 
          epochs=30, verbose=1, 
          validation_data=(x_valid, y_valid))

Epoch 1/30
186/186 [==============================] - 2s 5ms/step - loss: 2.1561 - categorical_accuracy: 0.3147 - f1_score: 0.1260 - val_loss: 1.6876 - val_categorical_accuracy: 0.3948 - val_f1_score: 0.1610
Epoch 2/30
186/186 [==============================] - 1s 4ms/step - loss: 1.3773 - categorical_accuracy: 0.4697 - f1_score: 0.2413 - val_loss: 1.5902 - val_categorical_accuracy: 0.4154 - val_f1_score: 0.1873
Epoch 3/30
186/186 [==============================] - 1s 4ms/step - loss: 1.2174 - categorical_accuracy: 0.5204 - f1_score: 0.3456 - val_loss: 1.5829 - val_categorical_accuracy: 0.4117 - val_f1_score: 0.1957
Epoch 4/30
186/186 [==============================] - 1s 4ms/step - loss: 1.1207 - categorical_accuracy: 0.5622 - f1_score: 0.4198 - val_loss: 1.6146 - val_categorical_accuracy: 0.4160 - val_f1_score: 0.1967
Epoch 5/30
186/186 [==============================] - 1s 4ms/step - loss: 1.0402 - categorical_accuracy: 0.5954 - f1_score: 0.4866 - val_loss: 1.6675 - val_categorical_